# 情感分析

## 在 SageMaker 中更新模型

_机器学习工程师纳米学位课程 | 开发_

---

在此 notebook 中，我们将考虑以下一种情形：我们构建的模型效果不如当初。我们将查看之前构建的 XGBoost 情感分析模型。但是，有一些新数据，模型似乎在这些新数据上效果不太好。所以，我们需要重新训练模型，并更新现有的端点，使其使用新的模型。

首先，我们将重新创建在之前的 notebook 中创建的 XGBoost 情感分析模型。所以你已经见过到第 4 步结束时的单元格。新内容从第 5 步开始。

## 说明

我们已经提供了一些模板代码，但是你需要实现其他功能，才能成功地完成此 notebook。除了要求的部分之外，不需要修改所包含的代码。标题以“**TODO**”开头的部分表示你需要完成或实现其中的某些部分。我们将在每个部分提供说明，并在代码块中用 `# TODO: ...` 注释标记出具体的实现要求。请务必仔细阅读说明。

除了实现代码之外，你还需要回答一些问题，这些问题与任务和你的实现代码有关。每个部分需要回答的问题都在标题中以“**问题：**”开头。请仔细阅读每个问题，并编辑下面以“**答案：**”开头的标记单元格，然后输入答案。

> 注意：可以通过 **Shift+Enter** 键盘快捷键执行代码和标记单元格。此外，通常还可通过点击单元格（标记单元格需要双击）编辑单元格，或者在选中后按下 **Enter** 键编辑单元格。

## 第 1 步：下载数据

我们要使用的数据集很受自然语言处理领域的研究者欢迎，通常称为 [IMDB 数据集](http://ai.stanford.edu/~amaas/data/sentiment/)。其中包含网站 [imdb.com](http://www.imdb.com/) 上的影评，每条影评都标有“**pos**itive”，表示评论者喜欢影片，否则标有“**neg**ative”。

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

我们先通过 Jupyter Notebook 功能下载和提取该数据集。

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## 第 2 步：准备数据

我们下载的文件拆分成了各种文件，每个都包含一条影评。我们需要将这些文件合并成两个文件，一个用于训练，一个用于测试。

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

In [ ]:
train_X[100]

## 第 3 步：处理数据

合并并准备好训练和测试数据集后，我们需要将原始数据处理成机器学习算法能够使用的格式。首先，删除所有的 HTML 格式标记，并执行一些标准自然语言处理步骤，使数据变得类同。

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
import re
from bs4 import BeautifulSoup

def review_to_words(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [ ]:
review_to_words(train_X[100])

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

### 提取词袋特征

对于我们要实现的模型，我们并不直接使用影评，而是将每条影评转换成词袋特征表示法。注意，我们只能访问训练集，所以转换器只能使用训练集创建表示结果。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [ ]:
# Extract Bag of Words features for both training and test datasets
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

In [ ]:
len(train_X[100])

## 第 4 步：使用 XGBoost 进行分类

创建了训练（和测试）数据的特征表示结果后，我们将开始设置和使用 SageMaker 提供的 XGBoost 分类器。

### 写入数据集

我们将使用的 XGBoost 分类器要求我们将数据集写入文件中并将文件存储到 Amazon S3 上。我们首先将训练数据集拆分成两部分，分别是训练集和验证集。然后，将这些数据集写入文件中，并将文件上传到 S3。此外，我们将测试集输入写入文件中并将文件上传到 S3。这样才能使用 SageMaker 批转换功能测试拟合后的模型。

In [ ]:
import pandas as pd

# Earlier we shuffled the training dataset so to make things simple we can just assign
# the first 10 000 reviews to the validation set and use the remaining reviews for training.
val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])

SageMaker 中的 XGBoost 算法的参考文档要求训练集和验证集不包含标题或索引，并且每个样本的标签在前面。

要详细了解此算法以及其他算法，请参阅 [Amazon SageMaker 开发人员文档](https://docs.aws.amazon.com/sagemaker/latest/dg/)。

In [ ]:
# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.
data_dir = '../data/sentiment_update'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [ ]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.

test_X = train_X = val_X = train_y = val_y = None

### (TODO) 将训练/验证文件上传到 S3

Amazon S3 服务允许我们存储文件，内置训练模型（例如我们将使用的 XGBoost 模型）和自定义模型（例如我们稍后将查看的模型）可以访问这些文件。

对于此任务以及将使用 SageMaker 完成的大多数其他任务，我们可以使用两种方法。一种是使用 SageMaker 的低阶方法，低阶方法要求我们知道在 SageMaker 环境中出现的每个对象。第二种是使用高阶方法，SageMaker 会代替我们做出一些选择。低阶方法的好处是给用户带来了很高的灵活性，而高阶方法使开发速度快多了。对我们来说，我们将使用高阶方法，但是也可以使用低阶方法。

方法 `upload_data()` 是代表当前 SageMaker 会话的对象的成员。该方法会将数据上传到默认存储桶（如果不存在的话，将会创建），并放入由 key_prefix 变量指定的路径下。上传数据文件后，你可以转到 S3 控制台并看看文件上传到哪了。

要查看其他资源，请参阅 [SageMaker API 文档](http://sagemaker.readthedocs.io/en/latest/)以及 [SageMaker 开发人员指南](https://docs.aws.amazon.com/sagemaker/latest/dg/)。

In [ ]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'sentiment-update'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

### 创建 XGBoost 模型


上传数据后，下面开始创建 XGBoost 模型。首先需要进行设置。此刻有必要讨论下模型在 SageMaker 中的含义。简单来说，可以将模型看作由 SageMaker 生态系统中的三个不同对象组成，它们相互交互。

- 模型工件
- 训练代码（容器）
- 推理代码（容器）

你可以将模型工件看作实际模型本身。例如，在构建神经网络时，可以将模型工件看作各个层级的权重。在我们的示例中，XGBoost 模型的模型工件是在训练过程中创建的实际树。

训练代码和推理代码将用来操纵模型工件。更准确地说，训练代码使用提供的训练数据并创建模型工件，而推理代码使用模型工件对新数据做出预测。

SageMaker 使用 Docker 容器运行训练和推理代码。暂时将容器看作一种代码打包方式，使依赖项不存在问题。

In [ ]:
from sagemaker import get_execution_role

# Our current execution role is require when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [ ]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost')

In [ ]:
# First we create a SageMaker estimator object for our model.
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

# And then set the algorithm specific parameters.
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

### 拟合 XGBoost 模型

设置好模型后，我们只需附加训练集和验证集，然后要求 SageMaker 设置计算过程。

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

### 测试模型

拟合 XGBoost 模型后，下面看看模型的效果如何。我们将使用 SageMaker 的批转换功能。通过批转换功能可以轻松地对大型数据集进行推理，因为它并非实时执行。我们并不需要立即使用模型的结果，可以对大量样本进行推理。示例行业应用包括月末报告。这种推理方法的另一个用处是可以对整个测试集进行推理。

为了执行批转换，我们首先需要根据训练过的 estimator 对象创建一个 transformer 对象。

In [ ]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

接下来执行转换作业。我们需要指定要发送的数据的类型，使 SageMaker 能够在后台正确地序列化数据。我们将向模型提供 csv 数据，所以指定为 `text/csv`。此外，如果我们提供的测试数据太大，无法一次性处理完，我们需要指定文件的拆分方式。因为数据集中的每行就是一个条目，所以我们将按照每行拆分输入数据。

In [ ]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

目前转换作业已经在运行，不过是在后台运行。因为我们要等待转换作业运行完毕，所以可以使用 `wait()` 方法查看运行进度。

In [ ]:
xgb_transformer.wait()

现在转换作业已经执行并且结果（每条影评的预测情感）已经保存到 S3 上。因为我们要在本地分析文件，所以通过一个 notebook 功能将文件复制到 `data_dir`。

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

最后一步是读入模型的输出，将输出转换成可用的格式，我们希望情感为 `1`（正面）或 `0`（负面），然后与真实标签进行比较。

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

## 第 5 步：查看新数据

现在我们有了我们认为效果很好的 XGBoost 情感分析模型。所以，我们将部署模型并在应用中使用该模型。

但是，在用户使用我们的应用期间，我们会定期记录用户提交的影评，从而对部署的模型进行质量控制。收集了足够的影评后，我们手动查看这些影评，判断它们是正面的还是负面的（除了手动检查之外，还有很多其他方式）。这么做是为了检查模型的效果如何。

In [ ]:
import new_data

new_X, new_Y = new_data.get_new_data()

**注意：**这个 notebook 的有趣之处是尝试了解新数据到底发生了什么，所以不要提前查看 `new_data` 模块。此外，`new_data` 模块假设之前在第 3 步创建的缓存依然存储在 `../cache/sentiment_analysis` 中。

### (TODO) 测试当前模型

加载了新数据后，我们看看当前 XGBoost 模型在新数据上的效果如何。

首先注意，加载的数据已经预处理过，所以 `new_X` 中的每个条目是使用 `nltk` 处理过的字词列表。但是，我们还没有创建词袋，下面将创建。

首先，使用之前使用原始训练数据创建的词汇表构建 `CountVectorizer`，我们将用它将新数据转换成词袋编码。

**TODO：**使用之前创建的词汇表构建 CountVectorizer 对象，并使用该对象转换新数据。

In [ ]:
# TODO: Create the CountVectorizer using the previously constructed vocabulary
vectorizer = None

# TODO: Transform our new data set and store the transformed data in the variable new_XV
new_XV = None

快速检查下每个词袋形式的影评长度是否正确。长度必须与词汇表的大小一样，即 `5000`。

In [ ]:
len(new_XV[100])

按照模型的要求处理了数据后，我们可以将数据保存到本地并上传到 S3，以便构建批转换作业并查看模型的效果。

首先将数据保存到本地。

**TODO：**将新数据（使用原始词汇表转换后）保存到本地 notebook 实例中。

In [ ]:
# TODO: Save the data contained in new_XV locally in the data_dir with the file name new_data.csv

接下来，将数据上传到 S3。

**TODO：**将上面创建的 csv 文件上传到 S3。

In [ ]:
# TODO: Upload the new_data.csv file contained in the data_dir folder to S3 and save the resulting
#       URI as new_data_location

new_data_location = None

将新数据上传到 S3 后，下面创建并运行批转换作业，使模型预测新影评的情感。

**TODO：**使用之前创建的 `xgb_transformer` 对象（在第 4 步测试 XGBoost 模型结束处）转换位于 `new_data_location` 的数据。

In [ ]:
# TODO: Using xgb_transformer, transform the new_data_location data. You may wish to **wait** until
#       the batch transform job has finished.

和之前一样，将批转换作业的结果复制到本地实例中。

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

读取批转换作业的结果。

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'new_data.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

检查当前模型的准确率。

In [ ]:
accuracy_score(new_Y, predictions)

看来某个方面出现了变化，因为模型在判断用户提交的影评的情感方面效果不如当初了。

在现实中，你需要检查多个方面，看看到底发生了什么。对我们来说，我们将仅检查一项内容，即底层数据分布是否发生了变化。换句话说，我们想检查新的影评集合中的字词是否与原始训练集中出现的字词相符。当然，我们需要缩小范围，只查看每个数据集中的前 `5000` 个字词，即每个数据集生成的词汇表。

在此之前，我们来看看新数据集中分类错误的某些影评。

首先，我们将部署原始 XGBoost 模型，然后使用部署的模型推理某些影评的情感。这样可以模拟实际生产场景，即在生产环境中使用原始模型的同时，对这个模型进行热更新。

**TODO：**部署 XGBoost 模型。

In [ ]:
# TODO: Deploy the model that was created earlier. Recall that the object name is 'xgb'.
xgb_predictor = None

### 诊断问题

部署了 "production" 模型后，我们可以向其发送一些新数据，并滤除某些分类错误的影评。

In [ ]:
from sagemaker.predictor import csv_serializer

# We need to tell the endpoint what format the data we are sending is in so that SageMaker can perform the serialization.
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

有必要查看几个不同的分类错误的影评示例，首先创建一个生成器，并用它遍历某些新的影评，查看哪些影评分类错误。

**注意：**在此模块中，并非必须了解什么是 Python 生成器。我们使用生成器的原因是，不需要遍历所有新的影评来搜索分类错误的样本。

In [ ]:
def get_sample(in_X, in_XV, in_Y):
    for idx, smp in enumerate(in_X):
        res = round(float(xgb_predictor.predict(in_XV[idx])))
        if res != in_Y[idx]:
            yield smp, in_Y[idx]

In [ ]:
gn = get_sample(new_X, new_XV, new_Y)

此刻，`gn` 是一个生成器，它会从新数据集中寻找分类错误的样本。要获得下个样本，只需对生成器调用 `next` 方法。

In [ ]:
print(next(gn))

查看了几个示例后，我们看看每个数据集（原始训练集和新的数据集）中的前 `5000` 个字词。目的是看看不同字词的使用频率是否改变了，也许出现了新的俚语，或者其他流行文化导致人们撰写影评的用词变了。

首先用 `CountVectorizer` 拟合新数据。

In [ ]:
new_vectorizer = CountVectorizer(max_features=5000,
                preprocessor=lambda x: x, tokenizer=lambda x: x)
new_vectorizer.fit(new_X)

创建了新的 `CountVectorizor` 对象后，我们看看两个数据集对应的词汇表之间是否有变化。

In [ ]:
original_vocabulary = set(vocabulary.keys())
new_vocabulary = set(new_vectorizer.vocabulary_.keys())

我们可以查看出现在原始词汇表中但是不在新词汇表中的字词。

In [ ]:
print(original_vocabulary - new_vocabulary)

同理，我们可以查看出现在新词汇表中但是不在原始词汇表中的字词。

In [ ]:
print(new_vocabulary - original_vocabulary)

这些字词本身并不能告诉我们什么，但是如果某个字词的出现频率很高，那么可能就有问题了。我们不希望上述任何字词的出现频率太高。

**问题：**到底发生了什么？哪些字词（如果有）的出现频率超出了预期？这意味着什么？原始模型不再考虑的字词出现了什么变化？

**注意：**这些都是开放式问题。要回答这些问题，下面提供的单元格可能还不够。并没有什么正确答案，只是希望你能借此机会了解下数据。

### (TODO) 构建新模型

假设我们认为组成影评的字词底层分布发生了变化，我们需要创建一个新的模型。这样的话，新模型就可以考虑到所发生的变化。

首先，我们将使用新词汇表创建新数据的词袋编码。然后，使用词袋编码训练新的 XGBoost 模型。

**注意：**因为我们认为底层字词分布改变了，所以用来构建影评词袋编码的原始词汇表应该不再有效了。我们需要谨慎地使用数据。如果发送使用原始词汇表创建的词袋编码，将不会获得任何有意义的结果。

如果我们像之前在网络应用 notebook 里部署 XGBoost 模型一样操作，那么还需要在 Lambda 函数中实现这种词汇表变化。

In [ ]:
new_XV = new_vectorizer.transform(new_X).toarray()

快速检查下新编码的影评是否长度正确，长度应该等于新创建的词汇表的大小。

In [ ]:
len(new_XV[0])

有了新编码、新收集的数据后，我们可以将其拆分为训练集和验证集，以便训练新的 XGBoost 模型。和之前一样，首先拆分数据，然后保存到本地，接着上传到 S3。

In [ ]:
import pandas as pd

# Earlier we shuffled the training dataset so to make things simple we can just assign
# the first 10 000 reviews to the validation set and use the remaining reviews for training.
new_val_X = pd.DataFrame(new_XV[:10000])
new_train_X = pd.DataFrame(new_XV[10000:])

new_val_y = pd.DataFrame(new_Y[:10000])
new_train_y = pd.DataFrame(new_Y[10000:])

为了节省内存，我们将删除 `new_X` 变量。此变量包含影评列表，每个影评都是一个字词列表。注意，执行了以下单元格后，如果你想使用新数据，需要重新读取新数据。

In [ ]:
new_X = None

接着将新的训练集和测试集保存到本地。注意，我们覆盖了之前使用的训练集和验证集。因为 notebook 实例的可用内存是有限的。当然，你也可以增加 notebook 实例的内存，但是这样可能会增加运行 notebook 实例的费用。

In [ ]:
pd.DataFrame(new_XV).to_csv(os.path.join(data_dir, 'new_data.csv'), header=False, index=False)

pd.concat([new_val_y, new_val_X], axis=1).to_csv(os.path.join(data_dir, 'new_validation.csv'), header=False, index=False)
pd.concat([new_train_y, new_train_X], axis=1).to_csv(os.path.join(data_dir, 'new_train.csv'), header=False, index=False)

将数据保存到本地实例中后，我们可以安全地删除变量并节省内存了。

In [ ]:
new_val_y = new_val_X = new_train_y = new_train_X = new_XV = None

最后，将新的训练集和验证集上传到 S3。

**TODO：**将新数据以及新的训练和验证集上传到 S3。

In [ ]:
# TODO: Upload the new data and the new validation.csv and train.csv files in the data_dir directory to S3.
new_data_location = None
new_val_location = None
new_train_location = None

将新的训练数据上传到 S3 后，我们可以创建新的 XGBoost 模型，它将考虑到数据集中出现的变化。

**TODO：**创建新的 XGBoost estimator 对象。

In [ ]:
# TODO: First, create a SageMaker estimator object for our model.
new_xgb = None

# TODO: Then set the algorithm specific parameters. You may wish to use the same parameters that were
#       used when training the original model.


创建模型后，我们可以使用新数据训练模型。

**TODO：**训练新的 XGBoost 模型。

In [ ]:
# TODO: First, make sure that you create s3 input objects so that SageMaker knows where to
#       find the training and validation data.
s3_new_input_train = None
s3_new_input_validation = None

In [ ]:
# TODO: Using the new validation and training data, 'fit' your new model.


### (TODO) 检查新模型

现在，我们有了新的 XGBoost 模型，我们认为它能更准确地代表目前的状况，至少在我们要解决的情感分析问题方面是这样的。下一步，我们将再次检查模型是否性能不错。

首先，我们用新数据测试模型。

**注意：**在实践中，这种做法很糟糕。我们已经用新数据训练了模型，所以用新数据测试并不能提供有效的信息。实际上，这属于典型的疏忽问题。我们这么做只是为了有一个数值基准。

**问题：**如何解决这个疏忽问题？

首先，根据新的 XGBoost 模型创建新的 transformer。

**TODO：**根据新创建的 XGBoost 模型创建新的 transformer 对象。

In [ ]:
# TODO: Create a transformer object from the new_xgb model
new_xgb_transformer = None

接着使用新数据测试模型。

**TODO：**使用 transformer 对象转换新数据（存储在 `new_data_location` 变量中）。

In [ ]:
# TODO: Using new_xgb_transformer, transform the new_data_location data. You may wish to
#       'wait' for the transform job to finish.


将结果复制到本地实例中。

In [ ]:
!aws s3 cp --recursive $new_xgb_transformer.output_path $data_dir

看看模型的效果。

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'new_data.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
accuracy_score(new_Y, predictions)

不出所料，因为我们用新数据训练了模型，所以模型的效果很好。我们有理由相信新的 XGBoost 模型效果更好。

但是，在开始更改部署的模型之前，我们需要首先确保新模型差别不是太大。换句话说，如果新模型在原始测试数据上的表现太差，那么表明可能其他方面出问题了。

首先，因为我们删除了存储原始测试影评的变量，所以需要从在第 3 步创建的缓存里重新读取测试影评。注意，我们需要在用 `nltk` 预处理数据之后且在用词袋编码之前读取原始测试数据。因为我们需要使用新的词汇表，而不是原始词汇表。

In [ ]:
cache_data = None
with open(os.path.join(cache_dir, "preprocessed_data.pkl"), "rb") as f:
            cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", "preprocessed_data.pkl")
            
test_X = cache_data['words_test']
test_Y = cache_data['labels_test']

# Here we set cache_data to None so that it doesn't occupy memory
cache_data = None

加载了原始测试影评后，我们需要使用根据新数据创建的新词汇表创建词袋编码。

**TODO：**使用新词汇表转换原始测试数据。

In [ ]:
# TODO: Use the new_vectorizer object that you created earlier to transform the test_X data.
test_X = None

正确编码原始测试数据后，我们可以将其写入本地实例中，并上传到 S3 上，然后进行测试。

In [ ]:
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

In [ ]:
test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)

In [ ]:
new_xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')
new_xgb_transformer.wait()

In [ ]:
!aws s3 cp --recursive $new_xgb_transformer.output_path $data_dir

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
accuracy_score(test_Y, predictions)

看来新的 XGBoost 模型在旧测试数据上表现很好。所以我们应该将新模型部署到生产环境中，并替换原始模型。

## 第 6 步：(TODO) 上传模型

我们有了新的可以使用的模型，而不是已经部署的模型。此外，我们假设已经部署的模型已经在某个应用中使用着。所以，我们希望更新现有端点，使其使用新的模型。

当然，我们需要为新创建的模型创建端点配置。

首先，注意我们可以使用 transformer 的 `model_name` 属性获取在上面创建的模型的名称。获取名称的原因是 transformer 创建批转换作业需要在 SageMaker 中创建模型对象。因为我们已经创建了模型对象，所以可以直接使用它。

In [ ]:
new_xgb_transformer.model_name

接着，我们使用创建字典对象的低阶方法创建端点配置，该字典对象描述了我们想要的端点配置。

**TODO：**利用低阶方法创建新的端点配置。端点配置需要一个唯一名称。如果遇到问题，请参阅 Boston Housing Low Level Deployment 教程 notebook。

In [ ]:
from time import gmtime, strftime


# TODO: Give our endpoint configuration a name. Remember, it needs to be unique.
new_xgb_endpoint_config_name = None

# TODO: Using the SageMaker Client, construct the endpoint configuration.
new_xgb_endpoint_config_info = None

创建了端点配置后，就要求 SageMaker 更新现有端点，使其使用新的端点配置。

注意，SageMaker 在完成这一步时不会中断服务。SageMaker 会部署新的模型，然后更新原始端点，使其指向新部署的模型。接着，关闭原始模型。这样的话，使用端点的应用就不会注意到我们已经更改了所使用的模型。

**TODO：**使用 SageMaker 客户端更新之前部署的端点。

In [ ]:
# TODO: Update the xgb_predictor.endpoint so that it uses new_xgb_endpoint_config_name.


与其他 SageMaker 请求一样，SageMaker 会在后台完成操作，如果我们想等待运行完毕，需要调用相应的方法。

In [ ]:
session.wait_for_endpoint(xgb_predictor.endpoint)

## 第 7 步：删除端点

使用完部署的端点后，我们需要关闭端点，否则会继续产生费用。

In [ ]:
xgb_predictor.delete_endpoint()

## 其他问题

此 notebook 与这一模块中的其他 notebook 有所不同。因为我们希望它能更接近你在现实中可能会遇到的问题。当然，这个问题很简单，已经具有解决方案，但是还有很多其他有趣的问题没有考虑，你可以自己思考一下这些问题。

例如：
- 底层分布还有哪些其他变化形式？
- 仅使用新数据重新训练模型合理吗？
- 如果新数据量不大，哪些会改变？例如仅收到 500 个样本。


## 可选步骤：清理数据

SageMaker 上的默认 notebook 实例没有太多的可用磁盘空间。当你继续完成和执行 notebook 时，最终会耗尽磁盘空间，导致难以诊断的错误。完全使用完 notebook 后，建议删除创建的文件。你可以从终端或 notebook hub 删除文件。以下单元格中包含了从 notebook 内清理文件的命令。

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we will remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir